# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [5]:
# target = ["loan_status"]

# Create our features
X = df.drop(columns=target)

# Create our target
y = df[target]

### encode, fit and transform

In [6]:
# Creating an instance of label encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# if needed, encode the column keep the schema.
# perform fit_transform on all.
encoded_keys = dict()
for feature in X.columns.values:
    X[feature] = label_encoder.fit_transform(df[feature])
    if ((df.dtypes[feature] != np.int64) and (df.dtypes[feature] != np.float64)):
        encoded_keys[feature] = label_encoder.classes_.tolist()
encoded_keys

{'home_ownership': ['ANY', 'MORTGAGE', 'OWN', 'RENT'],
 'verification_status': ['Not Verified', 'Source Verified', 'Verified'],
 'issue_d': ['Feb-2019', 'Jan-2019', 'Mar-2019'],
 'pymnt_plan': ['n'],
 'initial_list_status': ['f', 'w'],
 'next_pymnt_d': ['Apr-2019', 'May-2019'],
 'application_type': ['Individual', 'Joint App'],
 'hardship_flag': ['N'],
 'debt_settlement_flag': ['N']}

In [7]:
# trim constant features
trim_features = []
for feature in encoded_keys:
    if len(encoded_keys[feature]) == 1:
        trim_features.append(feature)
for feature in trim_features:
    encoded_keys.pop(feature, None)
X = X.drop(columns=trim_features)
trim_features

['pymnt_plan', 'hardship_flag', 'debt_settlement_flag']

In [8]:
# final features.
X.head(3)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,dti,delinq_2yrs,inq_last_6mths,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,348,24,3500,3,2978,1,2,2590,0,0,...,0,3,285,155,0,0,11675,19018,19,27490
1,886,28,7846,1,4688,2,2,1889,0,0,...,0,6,331,101,1,0,39916,30368,412,21865
2,703,28,4973,1,2339,2,2,2292,0,0,...,0,4,142,101,0,0,10267,23239,75,18964


In [9]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,dti,delinq_2yrs,inq_last_6mths,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000
mean,572.296903,12.982577,4126.615168,1.812779,3294.087856,0.669994,0.805542,1962.376855,0.217722,0.497697,...,0.052124,2.219408,367.820219,57.069314,0.125972,0.0,27710.412107,24184.046704,295.617696,19966.952715
std,375.779218,9.508701,2367.621789,0.941313,1574.576387,0.719105,0.714932,971.922012,0.717338,0.758122,...,0.389984,1.897307,67.053040,55.878931,0.336732,0.0,17546.728857,14497.653994,257.292302,13661.709989
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,289.000000,5.000000,2153.000000,1.000000,1930.000000,0.000000,0.000000,1255.000000,0.000000,0.000000,...,0.000000,1.000000,347.000000,0.000000,0.000000,0.0,12014.000000,11764.000000,116.000000,7895.000000
50%,516.000000,11.000000,3828.000000,1.000000,3365.000000,1.000000,1.000000,1842.000000,0.000000,0.000000,...,0.000000,2.000000,409.000000,48.000000,0.000000,0.0,26735.000000,23056.000000,221.000000,18429.000000
75%,848.000000,20.000000,5967.000000,3.000000,4665.000000,1.000000,1.000000,2532.000000,0.000000,1.000000,...,0.000000,3.000000,409.000000,101.000000,0.000000,0.0,42861.000000,36149.000000,393.000000,31243.000000
max,1397.000000,51.000000,8809.000000,3.000000,6089.000000,2.000000,2.000000,5695.000000,17.000000,5.000000,...,17.000000,18.000000,409.000000,155.000000,4.000000,0.0,59628.000000,51997.000000,1715.000000,46898.000000


In [10]:
y = df[['loan_status']]
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled['loan_status'].values)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
# testing data
y_pred = model.predict(X_test)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.7236405820189129

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

pd.DataFrame(data = confusion_matrix(y_test, y_pred),
            index = ['Actual 0', 'Actual 1'],
            columns = ['Predicted 0', 'Predicted 1'])

,Predicted 0,Predicted 1
Actual 0,73,28
Actual 1,4712,12392


In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.72      0.72      0.03      0.72      0.52       101
   low_risk       1.00      0.72      0.72      0.84      0.72      0.52     17104

avg / total       0.99      0.72      0.72      0.83      0.72      0.52     17205



### SMOTE Oversampling

In [20]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto')    \
            .fit_resample(X_train, y_train)

Counter(y_resampled['loan_status'].values)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [21]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.726388477248099

In [23]:
# Display the confusion matrix
pd.DataFrame(data = confusion_matrix(y_test, y_pred),
            index = ['Actual 0', 'Actual 1'],
            columns = ['Predicted 0', 'Predicted 1'])

,Predicted 0,Predicted 1
Actual 0,73,28
Actual 1,4618,12486


In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.72      0.73      0.03      0.73      0.53       101
   low_risk       1.00      0.73      0.72      0.84      0.73      0.53     17104

avg / total       0.99      0.73      0.72      0.84      0.73      0.53     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [25]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled['loan_status'].values)

Counter({'high_risk': 246, 'low_risk': 246})

In [26]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7130110842000945

In [28]:
# Display the confusion matrix
pd.DataFrame(data = confusion_matrix(y_test, y_pred),
            index = ['Actual 0', 'Actual 1'],
            columns = ['Predicted 0', 'Predicted 1'])

,Predicted 0,Predicted 1
Actual 0,77,24
Actual 1,5753,11351


In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.76      0.66      0.03      0.71      0.51       101
   low_risk       1.00      0.66      0.76      0.80      0.71      0.50     17104

avg / total       0.99      0.66      0.76      0.79      0.71      0.50     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled['loan_status'].values)

Counter({'high_risk': 68470, 'low_risk': 64463})

In [31]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.728737241708268

In [33]:
# Display the confusion matrix
pd.DataFrame(data = confusion_matrix(y_test, y_pred),
            index = ['Actual 0', 'Actual 1'],
            columns = ['Predicted 0', 'Predicted 1'])

,Predicted 0,Predicted 1
Actual 0,74,27
Actual 1,4707,12397


In [34]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.73      0.72      0.03      0.73      0.53       101
   low_risk       1.00      0.72      0.73      0.84      0.73      0.53     17104

avg / total       0.99      0.72      0.73      0.83      0.73      0.53     17205



## Results Analysis
### High Risk detection
We studied four sampling here:
- Naive random oversampling
- SMOTE oversampling
- Cluster Centroids undersampling
- SMOTEENN combined sampling   

Cluster Centroids undersampling gives the best chance to detect high risk credits (sensivity 76%), with a lower precision 1%.

In [35]:
index_name = ['Naive OS', 'SMOTE', 'Centroids', 'SMOTEENN']
column_name = ['Precision', 'Sensitivity', 'f1']
res = [[0.02, 0.72, 0.03],
       [0.02, 0.72, 0.03],
       [0.01, 0.76, 0.03],
       [0.02, 0.73, 0.03]]
hi_risk_df = pd.DataFrame(data = res,
                          index = index_name,
                          columns = column_name)
hi_risk_df

,Precision,Sensitivity,f1
Naive OS,0.02,0.72,0.03
SMOTE,0.02,0.72,0.03
Centroids,0.01,0.76,0.03
SMOTEENN,0.02,0.73,0.03


### Low Risk detection
On low risk detection, all four samplings yield similar results, with the Centroids sampling yields a lower sensitivity (66%). 

In [37]:
res = [[1, 0.72, 0.72],
       [1, 0.73, 0.84],
       [1, 0.66, 0.8],
       [1, 0.72, 0.84]]
pd.DataFrame(data = res,
             index = index_name,
             columns = column_name)

,Precision,Sensitivity,f1
Naive OS,1,0.72,0.72
SMOTE,1,0.73,0.84
Centroids,1,0.66,0.80
SMOTEENN,1,0.72,0.84


### Balanced Accuracy score
All four sampling have about same accuracy.

In [38]:
res = [0.723, 0.726, 0.713, 0.728]
pd.DataFrame(data=res,
             index = index_name,
             columns = ['Accuracy'])

,Accuracy
Naive OS,0.723
SMOTE,0.726
Centroids,0.713
SMOTEENN,0.728


### Recommendation
It is important to detect the high risk credits.  
All four sampling ways have >= 72% sensitivities to detect high risk credits, with the Cluster Centroids undersampling looking most promising (76%).  
With the given dataset, tweaking the feature sets would yield different results. So ultimately, more data may decide whether or not the Centroids is the superior way to detect the high risk credits.  